In [1]:
from google.colab import drive
drive.mount("/content/drive/")
import os
os.chdir("drive/MyDrive/data")

Mounted at /content/drive/


In [2]:
!ls

emotion_labels.json	    final_train_tweets.json  sentiment_test_tweet.json
emotion_test_tweet.json     fixed_train_tweets.json  title_conference.csv
final_test_candidates.json  model		     topic_labels.json
final_test_tweets.json	    outp		     topic_test_tweet0.json
final_train_labels.json     sentiment_labels.json    topic_test_tweet.json


In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.6MB/s 
     |████████████████████████████████| 3.3MB 21.7MB/s 
     |████████████████████████████████| 901kB 39.7MB/s 


In [4]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd
import numpy as np


In [5]:
df_tls = pd.read_json("final_train_labels.json", orient='records', dtype=False)
print(df_tls.shape)
df_tls.head()

(2000, 3)


,tweet_id,m_id,m_label
0,1345642006299594752,7,agree
1,1339975577701392385,4,no_stance
2,1345867896426278913,10,agree
3,1344512357427511297,1,not_relevant
4,1340568636201848832,8,disagree


In [6]:
label_dict = {'agree': 0, 'disagree': 1, 'no_stance': 2, 'not_relevant': 3}
label_dict_inverse = {v: k for k, v in label_dict.items()}
df_tls['label'] = df_tls.m_label.replace(label_dict)
df_tls.head()

,tweet_id,m_id,m_label,label
0,1345642006299594752,7,agree,0
1,1339975577701392385,4,no_stance,2
2,1345867896426278913,10,agree,0
3,1344512357427511297,1,not_relevant,3
4,1340568636201848832,8,disagree,1


In [7]:
df_tts = pd.read_json("final_train_tweets.json", orient='records', dtype=False)
df_tts.rename(columns={'id':'tweet_id'}, inplace = True)
print(df_tts.shape)
df_tts.head()

(1640, 2)


,tweet_id,text
0,1344053526951985152,Human immune system has a higher success rate ...
1,1340150211423952896,Covid Vaccine is the new version of population...
2,1340070783536812034,Do not take the COVID-19 vaccine. You will nev...
3,1343310998485094400,@stealthbadger @oneconnelly1 Something in the ...
4,1343303464697270273,"The Pope of Rome declares that ""the use of the..."


In [8]:
df_ols = pd.read_json("topic_labels.json", orient='records', dtype=False)
df_ols.rename(columns={'id':'tweet_id'}, inplace = True)
print(df_ols.shape)
df_ols.head()

(1640, 3)


,tweet_id,text,topic
0,1344053526951985152,Human immune system has a higher success rate ...,immunity
1,1340150211423952896,Covid Vaccine is the new version of population...,people die
2,1340070783536812034,Do not take the COVID-19 vaccine. You will nev...,vaccine
3,1343310998485094400,@stealthbadger @oneconnelly1 Something in the ...,people die
4,1343303464697270273,"The Pope of Rome declares that ""the use of the...",aborted fetus tissue


In [9]:
topic_dict = {"immunity": 0,
             "side effect": 1,
             "infertility": 2,
             "microchip control": 3,
             "people die": 4,
             "dna": 5,
             "aborted fetus tissue": 6,
             "bell's palsy": 7,
             "vaccine": 8}

In [10]:
df_ols['topic_id'] = df_ols.topic.replace(topic_dict)
df_ols.head()

,tweet_id,text,topic,topic_id
0,1344053526951985152,Human immune system has a higher success rate ...,immunity,0
1,1340150211423952896,Covid Vaccine is the new version of population...,people die,4
2,1340070783536812034,Do not take the COVID-19 vaccine. You will nev...,vaccine,8
3,1343310998485094400,@stealthbadger @oneconnelly1 Something in the ...,people die,4
4,1343303464697270273,"The Pope of Rome declares that ""the use of the...",aborted fetus tissue,6


In [11]:
df0 = pd.merge(df_tls, df_ols[['tweet_id', 'topic_id']], how='left', on=['tweet_id'])
df = pd.merge(df0, df_tts, how='left', on=['tweet_id'])
print(df.shape)
df.head()

(2000, 6)


,tweet_id,m_id,m_label,label,topic_id,text
0,1345642006299594752,7,agree,0,3,Foetal cells are used to make the Oxford coron...
1,1339975577701392385,4,no_stance,2,7,They were just impersonating Tyler Herro URL Q...
2,1345867896426278913,10,agree,0,1,@RMCpost @SitaramYechury The AstraZeneca vacci...
3,1344512357427511297,1,not_relevant,3,8,This will happen if you take the Covid-19 vacc...
4,1340568636201848832,8,disagree,1,6,The COVID vaccine will not contain a microchip...


In [12]:
df['input'] = df['m_id'].map(lambda x: str(x) + " ") + df['topic_id'].map(lambda x: str(x) + " ") + df['text']
df.head()

,tweet_id,m_id,m_label,label,topic_id,text,input
0,1345642006299594752,7,agree,0,3,Foetal cells are used to make the Oxford coron...,7 3 Foetal cells are used to make the Oxford c...
1,1339975577701392385,4,no_stance,2,7,They were just impersonating Tyler Herro URL Q...,4 7 They were just impersonating Tyler Herro U...
2,1345867896426278913,10,agree,0,1,@RMCpost @SitaramYechury The AstraZeneca vacci...,10 1 @RMCpost @SitaramYechury The AstraZeneca ...
3,1344512357427511297,1,not_relevant,3,8,This will happen if you take the Covid-19 vacc...,1 8 This will happen if you take the Covid-19 ...
4,1340568636201848832,8,disagree,1,6,The COVID vaccine will not contain a microchip...,8 6 The COVID vaccine will not contain a micro...


In [13]:
df['m_label'].value_counts()

not_relevant    783
agree           608
disagree        354
no_stance       255
Name: m_label, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_trn, X_val, y_trn, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_trn, 'data_type'] = 'trn'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['m_label', 'label', 'data_type']).count()

tweet_id  m_id  topic_id  text  input
m_label      label data_type                                       
agree        0     trn             426   426       426   426    426
                   val             182   182       182   182    182
disagree     1     trn             248   248       248   248    248
                   val             106   106       106   106    106
no_stance    2     trn             178   178       178   178    178
                   val              77    77        77    77     77
not_relevant 3     trn             548   548       548   548    548
                   val             235   235       235   235    235

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [16]:
encoded_data_trn = tokenizer.batch_encode_plus(
    df[df.data_type=='trn'].input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_trn = encoded_data_trn['input_ids']
attention_masks_trn = encoded_data_trn['attention_mask']
labels_trn = torch.tensor(df[df.data_type=='trn'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_trn = TensorDataset(input_ids_trn, attention_masks_trn, labels_trn)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

print("Before encoding.")
print(df.input.values[0:8])
print("After encoding.")
print(input_ids_trn.numpy()[0:2])
print(input_ids_val.numpy()[0:2])
len(dataset_trn), len(dataset_val)

Before encoding.
['7 3 Foetal cells are used to make the Oxford coronavirus vaccine. But they came from a foetus in 1973 - ABC News URL'
 '4 7 They were just impersonating Tyler Herro URL QT: "Four trial volunteers who got Pfizer\'s COVID-19 vaccine developed Bell\'s palsy - but FDA denies that the temporary facial paralysis was caused by the shot.\nURL URL"'
 '10 1 @RMCpost @SitaramYechury The AstraZeneca vaccine was tested but had serious side effects worse than having COVID. So tell me why inject these in the first place?'
 '1 8 This will happen if you take the Covid-19 vaccine. URL'
 "8 6 The COVID vaccine will not contain a microchip, but It will have U2's new album!!"
 "8 3 @Maddamecrafter @KATUNews A vaccine? Sure. Is covid propaganda too? If they wanted to do population control they'd let covid rage on. If they wanted to track you and collect data they'll use your smart phone."
 '8 8 So is the COVID-19 vaccine a cover for administering a micro-chip? Hmm 😒'
 '4 7 Nurse Gets Bell

(1400, 600)

In [17]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_trn, 
                              sampler=RandomSampler(dataset_trn), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    preds_sum, true_sum = 0, 0

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}  Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
        true_sum += len(y_preds[y_preds==label])
        preds_sum += len(y_true)
    
    total_accuracy = "%.4f" % (true_sum / preds_sum * 100)
    print(f"Total accuracy: {total_accuracy}\n")
    return float(total_accuracy)

In [19]:
epochs = 20

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [20]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [21]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [22]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'model/finetuned_BERT_epoch_x4_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.1709613122476756
Validation loss: 1.0438049685955049
F1 Score (Weighted): 0.47742836355909996



Epoch 2
Training loss: 0.9427977045738722
Validation loss: 0.9579375293944031
F1 Score (Weighted): 0.5923039295058872



Epoch 3
Training loss: 0.7803654041803368
Validation loss: 1.3255901694577188
F1 Score (Weighted): 0.5762070921932461



Epoch 4
Training loss: 0.591048208708482
Validation loss: 1.4480164714960848
F1 Score (Weighted): 0.6395146358035464



Epoch 5
Training loss: 0.4277958658053211
Validation loss: 1.584272822185885
F1 Score (Weighted): 0.6602419161928136


...


Epoch 16
Training loss: 0.04635717147831382
Validation loss: 2.713232151579323
F1 Score (Weighted): 0.6258871593063379



Epoch 17
Training loss: 0.04243509986368868
Validation loss: 2.92583103252291
F1 Score (Weighted): 0.6132382367660426



Epoch 18
Training loss: 0.02209986455850909
Validation loss: 2.742348590432193
F1 Score (Weighted): 0.63253750845341



Epoch 19
Training loss: 0.01676237561282671
Validation loss: 2.7580706599033147
F1 Score (Weighted): 0.6325380445172093



Epoch 20
Training loss: 0.01632850330876203
Validation loss: 2.767855175597506
F1 Score (Weighted): 0.6292073974362254



In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
best_dir = ""
highest_acc = 0
for epoch in tqdm(range(1, epochs+1)):
    print(f"Epoch {epoch}:")
    dir = f"model/finetuned_BERT_epoch_x4_{epoch}.model"
    model.load_state_dict(torch.load(dir, map_location=torch.device('cpu')))
    _, predictions, true_vals = evaluate(dataloader_validation)
    acc = accuracy_per_class(predictions, true_vals)
    if acc > highest_acc:
        highest_acc = acc
        best_dir = dir
        print("Temporary best.\n")
print(f"Best model: {best_dir}")
model.load_state_dict(torch.load(best_dir, map_location=torch.device('cpu')))

Epoch 1:
Class: agree  Accuracy: 135/182
Class: disagree  Accuracy: 5/106
Class: no_stance  Accuracy: 0/77
Class: not_relevant  Accuracy: 198/235
Total accuracy: 56.3333

Temporary best.

Epoch 2:
Class: agree  Accuracy: 131/182
Class: disagree  Accuracy: 50/106
Class: no_stance  Accuracy: 5/77
Class: not_relevant  Accuracy: 188/235
Total accuracy: 62.3333

Temporary best.

Epoch 3:
Class: agree  Accuracy: 62/182
Class: disagree  Accuracy: 69/106
Class: no_stance  Accuracy: 21/77
Class: not_relevant  Accuracy: 209/235
Total accuracy: 60.1667

Epoch 4:
Class: agree  Accuracy: 92/182
Class: disagree  Accuracy: 75/106
Class: no_stance  Accuracy: 26/77
Class: not_relevant  Accuracy: 195/235
Total accuracy: 64.6667

Temporary best.

Epoch 5:
Class: agree  Accuracy: 112/182
Class: disagree  Accuracy: 69/106
Class: no_stance  Accuracy: 29/77
Class: not_relevant  Accuracy: 188/235
Total accuracy: 66.3333

Temporary best.

Epoch 6:
Class: agree  Accuracy: 92/182
Class: disagree  Accuracy: 63/10

<All keys matched successfully>

In [25]:
df_tcs = pd.read_json("final_test_candidates.json", orient='records', dtype=False)
df_tcs['tweet_id'] = df_tcs['tweet_id'].astype('int64')
print(df_tcs.shape)
df_tcs.head()

(1211, 2)


,tweet_id,m_id
0,1344676492102479875,10
1,1340430332231880705,3
2,1340032994254475268,3
3,1340020067476582404,10
4,1339941977174003714,3


In [26]:
df_tts2 = pd.read_json("final_test_tweets.json", orient='records', dtype=False)
df_tts2.rename(columns={'id':'tweet_id'}, inplace = True)
df_tts2['tweet_id'] = df_tts2['tweet_id'].astype('int64')
print(df_tts2.shape)
df_tts2.head()

(1012, 2)


,tweet_id,text
0,1339950458480316416,COVID-19 was for introduced to gain control ov...
1,1344547117294096384,The Covid-19 vaccine will not only save lives ...
2,1340161792782188545,"@kxly4news So, a person would have equivalent ..."
3,1340803627926118400,If there is another more deadly strain of COVI...
4,1340060162967650305,"Of cooooorse they aren't URL QT: ""There are 10..."


In [27]:
df_ots = pd.read_json("topic_test_tweet.json", orient='records', dtype=False)
df_ots.rename(columns={'id':'tweet_id'}, inplace = True)
df_ots['tweet_id'] = df_ots['tweet_id'].astype('int64')
print(df_ots.shape)
df_ots.head()

(1012, 4)


,tweet_id,text,label,topic
0,1339950458480316416,COVID-19 was for introduced to gain control ov...,0,people die
1,1344547117294096384,The Covid-19 vaccine will not only save lives ...,0,vaccine
2,1340161792782188545,"@kxly4news So, a person would have equivalent ...",0,people die
3,1340803627926118400,If there is another more deadly strain of COVI...,0,side effect
4,1340060162967650305,"Of cooooorse they aren't URL QT: ""There are 10...",0,microchip control


In [28]:
df_ots['topic_id'] = df_ots.topic.replace(topic_dict)
df_ots.head()

,tweet_id,text,label,topic,topic_id
0,1339950458480316416,COVID-19 was for introduced to gain control ov...,0,people die,4
1,1344547117294096384,The Covid-19 vaccine will not only save lives ...,0,vaccine,8
2,1340161792782188545,"@kxly4news So, a person would have equivalent ...",0,people die,4
3,1340803627926118400,If there is another more deadly strain of COVI...,0,side effect,1
4,1340060162967650305,"Of cooooorse they aren't URL QT: ""There are 10...",0,microchip control,3


In [29]:
df20 = pd.merge(df_tcs, df_ots[['tweet_id', 'topic_id']], how='left', on=['tweet_id'])
df2 = pd.merge(df20, df_tts2, how='left', on=['tweet_id'])
df2['tweet_id'] = df2['tweet_id'].astype('object')
print(df2.dtypes)
print(df2.shape)
df2.head()

tweet_id    object
m_id        object
topic_id     int64
text        object
dtype: object
(1211, 4)


,tweet_id,m_id,topic_id,text
0,1344676492102479875,10,1,"Just so y'all know, it's possible that the vac..."
1,1340430332231880705,3,4,@JCope222 Vaccine is untested for safety. Peop...
2,1340032994254475268,3,0,@catturd2 Why would antibodies formed from rec...
3,1340020067476582404,10,1,I started thinking about the long term effects...
4,1339941977174003714,3,0,I like vaccine herd immunity over infection he...


In [30]:
df2['input'] = df2['m_id'].map(lambda x: str(x) + " ") + df2['topic_id'].map(lambda x: str(x) + " ") + df2['text']
df2['label'] = 0
df2.head()

,tweet_id,m_id,topic_id,text,input,label
0,1344676492102479875,10,1,"Just so y'all know, it's possible that the vac...","10 1 Just so y'all know, it's possible that th...",0
1,1340430332231880705,3,4,@JCope222 Vaccine is untested for safety. Peop...,3 4 @JCope222 Vaccine is untested for safety. ...,0
2,1340032994254475268,3,0,@catturd2 Why would antibodies formed from rec...,3 0 @catturd2 Why would antibodies formed from...,0
3,1340020067476582404,10,1,I started thinking about the long term effects...,10 1 I started thinking about the long term ef...,0
4,1339941977174003714,3,0,I like vaccine herd immunity over infection he...,3 0 I like vaccine herd immunity over infectio...,0


In [31]:
encoded_data_pred = tokenizer.batch_encode_plus(
    df2.input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_pred = encoded_data_pred['input_ids']
attention_masks_pred = encoded_data_pred['attention_mask']
labels_pred = torch.tensor(df2.label.values)  # pretend to be all 0s. Proved that labels does not change results.

dataset_pred = TensorDataset(input_ids_pred, attention_masks_pred, labels_pred)

In [32]:
dataloader_prediction = DataLoader(dataset_pred, 
                              sampler=SequentialSampler(dataset_pred),
                              batch_size=1)

In [33]:
def predict(dataloader_val):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
            
    return np.argmax(predictions, axis=1).flatten()

In [34]:
test_predictions = predict(dataloader_prediction)
df2['label'] = pd.Series(test_predictions).values
print(df2.shape)
df2.head()

(1211, 6)


,tweet_id,m_id,topic_id,text,input,label
0,1344676492102479875,10,1,"Just so y'all know, it's possible that the vac...","10 1 Just so y'all know, it's possible that th...",1
1,1340430332231880705,3,4,@JCope222 Vaccine is untested for safety. Peop...,3 4 @JCope222 Vaccine is untested for safety. ...,3
2,1340032994254475268,3,0,@catturd2 Why would antibodies formed from rec...,3 0 @catturd2 Why would antibodies formed from...,2
3,1340020067476582404,10,1,I started thinking about the long term effects...,10 1 I started thinking about the long term ef...,0
4,1339941977174003714,3,0,I like vaccine herd immunity over infection he...,3 0 I like vaccine herd immunity over infectio...,2


In [35]:
df2['m_label'] = df2.label.replace(label_dict_inverse)
df2.head()

,tweet_id,m_id,topic_id,text,input,label,m_label
0,1344676492102479875,10,1,"Just so y'all know, it's possible that the vac...","10 1 Just so y'all know, it's possible that th...",1,disagree
1,1340430332231880705,3,4,@JCope222 Vaccine is untested for safety. Peop...,3 4 @JCope222 Vaccine is untested for safety. ...,3,not_relevant
2,1340032994254475268,3,0,@catturd2 Why would antibodies formed from rec...,3 0 @catturd2 Why would antibodies formed from...,2,no_stance
3,1340020067476582404,10,1,I started thinking about the long term effects...,10 1 I started thinking about the long term ef...,0,agree
4,1339941977174003714,3,0,I like vaccine herd immunity over infection he...,3 0 I like vaccine herd immunity over infectio...,2,no_stance


In [36]:
import json
with open("outp/final_candidate_labels_x4.json", mode='w', encoding='utf-8') as f:
    for elem in json.loads(df2[['tweet_id', 'm_id', 'm_label']].to_json(orient='records')):
        line = json.dumps(elem) + "\n"
        f.write(line)